# Refine txt2img Prompts with Human Feedback


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CarperAI/trlx/blob/main/examples/notebooks/trlx_simulacra.ipynb)


#### Optimize a gpt2-based txt2img prompt generator to produce aesthetic prompts using a dataset of (prompt, rating) pairs https://github.com/JD-P/simulacra-aesthetic-captions

Notebook by [@smellslikeml](https://github.com/smellslikeml)

---

Execute the cells below to install [TRLX](https://github.com/CarperAI/trlx) for a colab environment.

In [1]:
!pip install git+https://github.com/CarperAI/trlx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CarperAI/trlx to /tmp/pip-req-build-hw1pzhf3
  Running command git clone --filter=blob:none --quiet https://github.com/CarperAI/trlx /tmp/pip-req-build-hw1pzhf3
  Resolved https://github.com/CarperAI/trlx to commit 65aafddbc4a7ad63e380e5ef22b8df56cc1efbfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.8/808.8 kB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:

In [1]:
!!pip install accelerate==0.16.0

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting accelerate==0.16.0',
 '  Downloading accelerate-0.16.0-py3-none-any.whl (199 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/199.7 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b199.7/199.7 kB\x1b \x1b10.4 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from accelerate==0.16.0) (1.24.3)',
 'Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.10/dist-packages (from accelerate==0.16.0) (23.1)',
 'Requirement already satisfied: psutil in /usr/local/lib/python3.10/dist-packages (from accelerate==0.16.0) (5.9.5)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/python3.10/dist-packages (from accelerate==0.16.0) (6.0)',
 'Requirement already satisfied: torch>=1.4.0 in /usr/local/li

In [2]:
import os
import sqlite3
from urllib.request import urlretrieve

import trlx

url = "https://raw.githubusercontent.com/JD-P/simulacra-aesthetic-captions/main/sac_public_2022_06_29.sqlite"
dbpath = "sac_public_2022_06_29.sqlite"

if not os.path.exists(dbpath):
  print(f"fetching {dbpath}")
  urlretrieve(url, dbpath)

conn = sqlite3.connect(dbpath)
c = conn.cursor()
c.execute(
    "SELECT prompt, rating FROM ratings "
    "JOIN images ON images.id=ratings.iid "
    "JOIN generations ON images.gid=generations.id "
    "WHERE rating IS NOT NULL;"
)

prompts, ratings = tuple(map(list, zip(*c.fetchall())))

[2023-06-16 05:39:54,273] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
fetching sac_public_2022_06_29.sqlite


In [ ]:
prompts

In [3]:
from trlx.data.default_configs import default_ilql_config
config = default_ilql_config().evolve(train=dict(batch_size=32, total_steps=300))

Trlx uses [wandb](https://wandb.ai/) to log results. Make sure to set up an account and use your token to authenticate when prompted after executing the cell below.

In [4]:
model = trlx.train(
    "gpt2",
    config=config,
    samples=prompts,
    rewards=ratings,
    eval_prompts=["<|endoftext|>"] * 64
).model

[RANK 0] Initializing model: gpt2


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[RANK 0] Collecting rollouts
[RANK 0] Logging sample example


                                              Sample Example                                              
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt        ┃ Response                                                                      ┃ Reward ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ <|endoftext|> │ An artwork of a broken wine bottle in the medium of dry pigments<|endoftext|> │ 7      │
└───────────────┴───────────────────────────────────────────────────────────────────────────────┴────────┘

...

                                                   Evaluation #2                                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt ┃ output                                                                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        │ A beautiful beautiful portrait headshot of a man with a beard looking into the sky, in the anime style │
│        │ of the anime.                                                                                          │
├────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│        │ concept art A painting of an angry man who has been attacked by aliens                                 │
├────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│        │ A portrait headshot of a man with the face of a lion, futuristic, oil paints.                          │
└────────┴────────────────────────────────────────────────────────────────────────────────────────────────────────┘

[RANK 0] Saving intermediate checkpoint into ckpts/checkpoint_300
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/2]:   0%|          | 0/2 [00:00<?, ?it/s]

[RANK 0] Summarizing evaluation


                                                   Evaluation #3                                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt ┃ output                                                                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        │ A painting of a man dressed as a gothic sorcerer with a hood. It is the most realistic and beautiful   │
│        │ of the magical spells used in the world to spell the night.                                            │
├────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│        │ portrait of the beautiful golden lady of Japan by Moebius by Jean Giraud Moebius by Thomas Kinkade     │
│        │ Thomas Wolfe Wolfe Von Linkiewicz                                                                      │
├────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│        │ An alien creature that looks like it's crawling through a city, artstation, by Greg Rutkowski and      │
│        │ David Heskin, 8k resolution, realistic, featured on ArtStation                                         │
└────────┴────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
# Infer the trained model
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
output = model.generate(**tokenizer(["An astronaut riding a horse"] * 16, return_tensors="pt").to(0))
tokenizer.batch_decode(output, skip_special_tokens=True)

['An astronaut riding a horse, an astronaut on a mountain',
 'An astronaut riding a horse, beautiful portrait, detailed digital art by Artgerm, wlop, Anton Fadeev, rossdraws, Ruan Jia, John',
 'An astronaut riding a horse, character design, realism, artstation, in the style sci-fi illustration, inspired by anime.',
 'An astronaut riding a horse by oskar schlemmer',
 'An astronaut riding a horse through the night on the cover of her book',
 'An astronaut riding a horse, beautiful portrait, detailed digital art by Artgerm, wlop, Anton Fadeev, Ruan Jia, John Conrad Berkey and Robert',
 'An astronaut riding a horse, a space cowboy riding a rocket, an astronaut, an astronaut wearing a sports gear, and a beautiful portrait of an astronaut.',
 'An astronaut riding a horse, painting by Anton Fadeev, featured on artstation',
 'An astronaut riding a horse, 4K digital illustration by John Berkey and James Gurney with Peter Morhbacher, John Berkey, Ross Tran, Artstation,',
 'An astronaut riding a 

In [ ]:
# Save the model locally
model.save_pretrained("gpt2-simulacra")

In [ ]:
# To upload the model to Hugging Face, login first
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Upload the model to <your_name>/gpt2-simulacra
from huggingface_hub import create_repo, HfApi

repo_id = create_repo("gpt2-simulacra", private=False, exist_ok=True).repo_id
HfApi().upload_folder(folder_path="gpt2-simulacra", repo_id=repo_id)

pytorch_model.bin:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

'https://huggingface.co/reciprocate/gpt2-simulacra/tree/main/'

In [ ]:
# Load the same model now stored on Hugging Face
from trlx.models.modeling_ilql import AutoModelForCausalLMWithILQLHeads
hf_model = AutoModelForCausalLMWithILQLHeads.from_pretrained(repo_id)

Some weights of the model checkpoint at reciprocate/gpt2-simulacra were not used when initializing GPT2LMHeadModel: ['ilql_heads.target_q_heads.1.2.bias', 'ilql_heads.q_heads.0.2.bias', 'ilql_heads.target_q_heads.0.2.bias', 'ilql_heads.q_heads.0.0.bias', 'ilql_heads.v_head.2.weight', 'ilql_heads.q_heads.1.0.bias', 'ilql_heads.v_head.0.bias', 'ilql_heads.target_q_heads.0.0.bias', 'ilql_heads.target_q_heads.1.0.bias', 'ilql_heads.v_head.0.weight', 'ilql_heads.v_head.2.bias', 'ilql_heads.q_heads.0.2.weight', 'ilql_heads.target_q_heads.1.2.weight', 'ilql_heads.q_heads.1.2.weight', 'ilql_heads.target_q_heads.0.0.weight', 'ilql_heads.target_q_heads.1.0.weight', 'ilql_heads.q_heads.0.0.weight', 'ilql_heads.q_heads.1.2.bias', 'ilql_heads.target_q_heads.0.2.weight', 'ilql_heads.q_heads.1.0.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model